# Traveling Salesperson Problem (TSP) with KOPPU

This notebook demonstrates solving the TSP using the KOPPU OPU. The goal is to find the shortest route visiting each city exactly once and returning to the origin.

## 1. Problem Formulation

Given a set of $N$ cities and distances $d_{ij}$ between them, we want to minimize the total tour length:

$$ L = \sum_{i,j} d_{ij} \sum_{t=1}^{N} x_{i,t} x_{j,t+1} $$

Subject to constraints:
1.  **Each city visited exactly once**: $\sum_{t=1}^{N} x_{i,t} = 1$ for all $i$.
2.  **Each time step has exactly one city**: $\sum_{i=1}^{N} x_{i,t} = 1$ for all $t$.

Where $x_{i,t} \in \{0, 1\}$ is 1 if city $i$ is visited at step $t$.

In [ ]:
import pykoppu as pk
import numpy as np

# 1. Define Cities and Distances
n_cities = 5
# Generate random distance matrix (symmetric)
np.random.seed(42)
coords = np.random.rand(n_cities, 2)
dist_matrix = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        dist_matrix[i, j] = np.linalg.norm(coords[i] - coords[j])

penalty = 5.0

## 2. PUBO Mapping

We map the constrained optimization to a PUBO by adding penalty terms for the constraints.

The Hamiltonian $H$ is:

$$ H = A \sum_{i} \left(\sum_{t} x_{i,t} - 1\right)^2 + A \sum_{t} \left(\sum_{i} x_{i,t} - 1\right)^2 + \sum_{i,j} d_{ij} \sum_{t} x_{i,t} x_{j,t+1} $$

Where $A$ is the penalty strength.

### Tensors and Solver Process

1.  **Tensors**: The constraints generate inhibitory couplings (penalty for multiple cities at same step or same city at multiple steps) and biases. The distance term generates couplings between variables representing adjacent steps ($t, t+1$).
2.  **Solver Process**: The OPU relaxes to a state that satisfies the constraints (valid tour) while minimizing the distance energy.

In [ ]:
# 2. Create TSP Problem
print("Mapping to OPU...")
problem = pk.problems.logistics.TSP(dist_matrix, penalty)

# 3. Solve
print("Solving...")
process = pk.oos.Process(problem, backend='cpu', t=500)
result = process.run()

# 4. Display OPU Dynamics
print("Visualizing OPU Dynamics...")
result.plot()

## 3. Solution Analysis

We visualize the tour found by the OPU.

In [ ]:
# 5. Analyse Solution
print("Visualizing Tour...")
problem.plot(result, threshold=0.5)